In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df_France_Official_Data2 = pd.read_csv("df_France_Official_Data_month.csv", index_col=0, header=0, squeeze=True, parse_dates=[0])
df_France_Official_Data2['date'] = pd.to_datetime(df_France_Official_Data2['date'])
df_France_Official_Data2

,date,department,value_in_thousands
0,2011-01-01,Ain,47.32
1,2011-01-01,Aisne,21.78
2,2011-01-01,Allier,23.62
3,2011-01-01,Alpes-Maritimes,170.80
4,2011-01-01,Alpes-de-Haute-Provence,15.27
...,...,...,...
12475,2021-10-01,Vendée,82.39
12476,2021-10-01,Vienne,96.87
12477,2021-10-01,Vosges,39.99
12478,2021-10-01,Yonne,41.35


In [77]:
#création de la liste contenant tous les départements pour itérer ensuite
liste_departement1 = df_France_Official_Data2['department'].tolist()
liste_departement = list(set(liste_departement1)) #on enlève tous les doublons
liste_departement = sorted(liste_departement) #on trie alphabétiquement la liste
len(liste_departement)
#liste_departement

96

## Interpolation sur 1 département

In [4]:
df_Official_Data_Sud = df_France_Official_Data2[(df_France_Official_Data2['department']=='Bouches-du-Rhône')] 
df_Official_Data_Sud
essai = df_Official_Data_Sud.drop('department', axis=1)
essai

,date,value_in_thousands
12,2011-01-01,153.08
108,2011-02-01,156.19
204,2011-03-01,216.20
300,2011-04-01,258.21
396,2011-05-01,294.41
...,...,...
12012,2021-06-01,280.21
12108,2021-07-01,415.19
12204,2021-08-01,427.94
12300,2021-09-01,334.81


In [5]:
upsampled = essai.set_index('date').resample('1D')
interpolated = upsampled.interpolate(method='linear')
interpolated

,value_in_thousands
date,
2011-01-01,153.080000
2011-01-02,153.180323
2011-01-03,153.280645
2011-01-04,153.380968
2011-01-05,153.481290
...,...
2021-09-27,314.781333
2021-09-28,314.011000
2021-09-29,313.240667


## Itération sur tous les départements

In [63]:
def monthtoday (nom_du_département):
    df_Official_Data_Sud = df_France_Official_Data2[(df_France_Official_Data2['department'] == nom_du_département)] 
    essai = df_Official_Data_Sud.drop('department', axis=1)
    upsampled = essai.set_index('date').resample('1D')
    interpolated = upsampled.interpolate(method='linear')

    return interpolated


In [71]:
# on initialise avec une dataframe vide
df = pd.DataFrame()
#on crée la boucle pour ajouter les valeurs pour chaque département dans le dataframe global
for i in liste_departement :
    x= str(i)
    y = monthtoday(x) #j'applique la fonction
    y['department']= x #on rajoute la colonne du département
    df = pd.concat([df,y]) # je concatène le dataframe de chaque département
    

In [64]:
df

,value_in_thousands,department,date
2011-01-01,47.320000,Ain,2011-01-01
2011-01-02,47.532258,Ain,2011-01-02
2011-01-03,47.744516,Ain,2011-01-03
2011-01-04,47.956774,Ain,2011-01-04
2011-01-05,48.169032,Ain,2011-01-05
...,...,...,...
2021-09-27,92.827333,Yvelines,2021-09-27
2021-09-28,93.128000,Yvelines,2021-09-28
2021-09-29,93.428667,Yvelines,2021-09-29
2021-09-30,93.729333,Yvelines,2021-09-30


In [72]:
df['date'] = df.index
df.index.name = None
df.reset_index()

,index,value_in_thousands,department,date
0,2011-01-01,47.320000,Ain,2011-01-01
1,2011-01-02,47.532258,Ain,2011-01-02
2,2011-01-03,47.744516,Ain,2011-01-03
3,2011-01-04,47.956774,Ain,2011-01-04
4,2011-01-05,48.169032,Ain,2011-01-05
...,...,...,...,...
376987,2021-09-27,92.827333,Yvelines,2021-09-27
376988,2021-09-28,93.128000,Yvelines,2021-09-28
376989,2021-09-29,93.428667,Yvelines,2021-09-29
376990,2021-09-30,93.729333,Yvelines,2021-09-30


In [74]:
donnees_off_interpolees= df.sort_values(by=['date', 'department'])
donnees_off_interpolees1 = donnees_off_interpolees.reset_index(drop=True)
donnees_off_interpolees1

,value_in_thousands,department,date
0,47.32,Ain,2011-01-01
1,21.78,Aisne,2011-01-01
2,23.62,Allier,2011-01-01
3,170.80,Alpes-Maritimes,2011-01-01
4,15.27,Alpes-de-Haute-Provence,2011-01-01
...,...,...,...
376987,82.39,Vendée,2021-10-01
376988,96.87,Vienne,2021-10-01
376989,39.99,Vosges,2021-10-01
376990,41.35,Yonne,2021-10-01


In [76]:
donnees_off_interpolees.to_csv("donnees_off_interpolees.csv")